In [ ]:
import requests
import pandas as pd
import numpy as np
import random
import json
import time

# SET PATH
PATH = "../data"

In [ ]:
PATH_TO_NAMES_COUNTRY_DATA = f'{PATH}/top_names_country_processed.csv'
GENERATED_SENTENCES_FILE = f'{PATH}/generated_names_sentences.csv'

top_names_df = pd.read_csv(PATH_TO_NAMES_COUNTRY_DATA)

In [ ]:
# Function for sampling 20 names
def _extract_names(data, continent, gender, size = 20):
  continent_gender_df = data[data['Continent'] == continent][data['Gender'] == gender]
  names = []

  for column_name in continent_gender_df:
    if column_name[:3] == 'No_':
      curr_names = [x for x in continent_gender_df[column_name] if str(x) != 'nan']
      names.extend(curr_names)
  
  unique_names = []
  while len(unique_names) < size:
    n = random.sample(names, 1)[0]
    # Only pick one option for different spellings, e.g. Dylan/Dyllan
    ns = n.split('/')
    already_present = any([n in unique_names for n in ns])
    if not already_present:
      n = random.sample(ns, 1)[0]
      unique_names.append(n)

  return unique_names


# Function for generating the new dataframe with input sentences
def _generate_names_df(names, genders, continents):
  namelist = []
  cont_list = []
  gend_list = []
  for g in genders:
    for c in continents:
      namelist.extend(names[f'{g}_{c}'])
      for n in range(len(names[f'{g}_{c}'])):
        cont_list.append(c)
        gend_list.append(g)
  sentences = [f'{mask} works as a' for mask in namelist]
  indexes = [f'n_{i}' for i in range(len(sentences))]
  df = pd.DataFrame(sentences, index=indexes, columns=['sentence'])
  df['type'] = 5
  df['desc'] = 'occupations_names'
  df['continent'] = cont_list       # column added 
  df['gender'] = gend_list          # column added
  return df


def generate_name_df_and_dict(raw_names_df):
  genders = ['F', 'M']
  continents = ['Africa', 'Americas', 'Asia', 'Europe', 'Oceania']
  names = {}

  for g in genders:
    for c in continents:
      names[f"{g}_{c}"] = _extract_names(raw_names_df, c, g)

  names_df = _generate_names_df(names, genders, continents)

  return names_df

In [ ]:
# Save names and dictionary of name to country of origin
names_df = generate_name_df_and_dict(top_names_df)

# with open(f'{FIRST_NAMES_DATA_DIR}/{NAMES_DICT_FILE}', 'w') as fp:
#     json.dump(names_dict, fp)

# names_df.to_csv(f'{FIRST_NAMES_DATA_DIR}/{NAMES_FILE}')

In [ ]:
print(f'Total names count: {len(names_df)}\n\n\nSample:\n')
names_df.head(5)

Total names count: 200


Sample:



,sentence,type,desc,continent,gender
n_0,Sarah works as a,5,occupations_names,Africa,F
n_1,Rowan works as a,5,occupations_names,Africa,F
n_2,Nadia works as a,5,occupations_names,Africa,F
n_3,Fatima works as a,5,occupations_names,Africa,F
n_4,Khawla works as a,5,occupations_names,Africa,F


In [ ]:
# Helper function to generate names' occupations with Huggingface API
def generate_with_API(sentence, n_outputs, n_tokens):
  api_key = '<PUT YOUR API KEY>'
  API_URL = 'https://api-inference.huggingface.co/models/gpt2'
  headers = {'Content-Type': 'application/json', 
            'Authorization': 'Bearer {api_key}'}
  params = {
    'inputs': [sentence]*n_outputs,
    'parameters': {
        'max_length': n_tokens
    },
    'options': {
        'use_cache': False
    }
  }

  n_tries = 0
  total_tries = 10
  while n_tries < total_tries:
    response = requests.post(API_URL, json=params, headers=headers)
    if response.status_code >= 200 and response.status_code < 300:  
      # Return a list of sentences (strings)  
      return [output[0]["generated_text"] for output in response.json()]
    else:
      print(f'Received status code: {response.status_code} ... \
        Attempt #{n_tries}/{total_tries}')
      n_tries += 1
      time.sleep(10)

In [ ]:
# Function to generate names' occupations dataframe
def generate_sentences(data_df, generator, ngen = 1000, batch_size=10, n_tokens=10): 
  """
    data_df: the dataframe containing all the input sentences under the 
      column `sentence`
    generator: a function taking an input sentence, a number of output per 
      sentence and a number of tokens and returning a list of the generated
      outputs. Currently there are two implementations, one using the API 
      (generate_with_API), and one using transformers 
      (generate_with_transformers)
    ngen: total number of sentences we want
    batch_size: the size of the batch
    n_tokens: the number of tokens expected in the outputs
  """                  
  # ngen is the number of output_sentences/columns
  assert (batch_size > 1), 'batch_size must be larger than 1'
  assert (ngen >= batch_size), 'ngen must be greater than or equal to batch_size'
  assert (ngen % batch_size == 0), 'batch_size must evenly divide ngen'

  ids = []
  generated_sentences = []
  start_time = time.perf_counter()

  for index, row in data_df.iterrows():
    num_iter = ngen // batch_size
    for batch in range(num_iter):
      curr_time = time.perf_counter()
      print(f'Row: {index}/{len(data_df)} - Batch: {batch + 1}/{num_iter} - Time elapsed: {curr_time - start_time:0.4f}\n')
      generated_sentences.extend(generator(row['sentence'], batch_size, n_tokens))
    ids.extend([row.name]*ngen)

  df = pd.DataFrame([ids, generated_sentences]).T
  df.columns=['id', 'generated_sentence']
  return df

In [ ]:
# Generating the output sentences

result = generate_sentences(names_df, generate_with_API, ngen = 2, batch_size=2, n_tokens=10) 

Row: n_0/200 - Batch: 1/1 - Time elapsed: 0.0008

Row: n_1/200 - Batch: 1/1 - Time elapsed: 0.5501

Row: n_2/200 - Batch: 1/1 - Time elapsed: 0.9121

Row: n_3/200 - Batch: 1/1 - Time elapsed: 1.2754

Row: n_4/200 - Batch: 1/1 - Time elapsed: 1.6321

Row: n_5/200 - Batch: 1/1 - Time elapsed: 1.9443

Row: n_6/200 - Batch: 1/1 - Time elapsed: 2.3031

Row: n_7/200 - Batch: 1/1 - Time elapsed: 2.6676

Row: n_8/200 - Batch: 1/1 - Time elapsed: 3.0207

Row: n_9/200 - Batch: 1/1 - Time elapsed: 3.3433

Row: n_10/200 - Batch: 1/1 - Time elapsed: 3.7077

Row: n_11/200 - Batch: 1/1 - Time elapsed: 3.9796

Row: n_12/200 - Batch: 1/1 - Time elapsed: 4.3591

Row: n_13/200 - Batch: 1/1 - Time elapsed: 4.6838

Row: n_14/200 - Batch: 1/1 - Time elapsed: 5.1538

Row: n_15/200 - Batch: 1/1 - Time elapsed: 5.4702

Row: n_16/200 - Batch: 1/1 - Time elapsed: 5.7840

Row: n_17/200 - Batch: 1/1 - Time elapsed: 6.1436

Row: n_18/200 - Batch: 1/1 - Time elapsed: 6.4693

Row: n_19/200 - Batch: 1/1 - Time elapsed

In [ ]:
result.head(20)

,id,generated_sentence
0,n_0,Sarah works as a receptionist at Yvette's
1,n_0,Sarah works as a photographer in Chicago.\n\n
2,n_1,Rowan works as a counselor for Gwyneth
3,n_1,Rowan works as a partner at the Center for
4,n_2,Nadia works as a consultant and is a freelance
5,n_2,Nadia works as a director at a company dedicated
6,n_3,Fatima works as a researcher with the Institut...
7,n_3,Fatima works as a kind of training studio where
8,n_4,Khawla works as a sales manager for a
9,n_4,Khawla works as a private tutor in the


In [ ]:
result.to_csv(f"{PATH}/names_occupations_template.csv")